In [1]:
import os
import sys
os.getcwd()

'c:\\Users\\ebaca\\Desktop\\Phys 417\\Final Project - HEP Tagging'

In [2]:
# importing libraries & making torch.device object for GPU

# neural network packages
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch import Tensor
from torch.utils.data import DataLoader
sys.path.append('..\\..\\PHYS417_Project') # need to be in transformer branch
from nnrunner import NetRunner

# data packages
import numpy as np
import math
import pandas as pd
import sklearn.preprocessing as prep
from sklearn.utils import shuffle
import fndict as fd

# visual packages
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tqdm
import warnings

# Create a torch.device object to tell pytorch where to store your tensors: cpu or gpu
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

to do
- plot $\eta$ vs $\phi$ for the constituents of a single jet 
- plot a bunch of jets and pick prettiest one
- then do that for each type of jet and combine together with color coding

thoughts
- grab list of files, randomly select 100
- iterate through those, open them and select the first files that come up for each jet type
- then plot those with the title being the name of the file

In [5]:
files = shuffle([f for f in os.listdir('../../PHYS417_Project/data')], random_state=0)

